In [2]:
import pickle
from os.path import exists
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import bitstring
import cv2 
from glob import glob

In [47]:
# Code created by Ferhat Özgur Catak. 
# Based on paper: 5G-SRNG: 5G Spectrogram-based Random Number
# Generation for Devices with Low Entropy Sources
files = glob('..\GeneratedData\\NR*.png')

num_of_files = len(files)

def get_entropy_spectrum(c, k, sel_img_idx, rng):
    sel_img = files[sel_img_idx]
    img = cv2.imread(sel_img)
    start_x = rng.integers(img.shape[0] - c)
    start_y = rng.integers(img.shape[1] - k)


    selected_spectogram = img[start_x:start_x+c, start_y:start_y+k, :].flatten()
    
    random_bits = bitstring.BitArray(float=selected_spectogram[0], length=32)
    
    for i in range(1,selected_spectogram.shape[0]):
        tmp = bitstring.BitArray(float=selected_spectogram[i], length=32) 
        random_bits = random_bits ^ tmp # xor
        random_bits = random_bits ^ random_bits << 13
        random_bits = random_bits ^ random_bits >> 17
        random_bits = random_bits ^ random_bits << 5
    
    return random_bits.uintle, random_bits


In [5]:
rng = np.random.default_rng(12345)
# entropies = [[[]*num_of_files]*170]*370
# entropies = [[[get_entropy_spectrum(c,k,s, rng) for s in range(num_of_files)] for k in range(1,169)] for c in range(1,369)]
# entropies = [[[get_entropy_spectrum(c,k,s, rng) for s in range(num_of_files)] for k in range(1,20)] for c in range(1,20)]
entropies = []
# # entropies = [entropies]
# for c in tqdm(range(1,369)):
for c in tqdm(range(1,30)):
    for k in range(1,30):
        temp = []
        for s in range(num_of_files):
            temp.append(get_entropy_spectrum(c,k,s, rng))
        entropies.append(temp)

Tried running with list comprehensions and without to see which is faster since it takes a very long time to run.  
List comprehensions with 10: 1m 15.7s   
Regular loop with 10: 1m 0.8s  
List comprehensions with 20: 18m 9.8s  
Regular loop with 20: 18m 18.4s  

In [46]:
rng = np.random.default_rng(12345)
for i in range(20):
    # print(get_entropy_spectrum(1,1,i,rng))
    print(get_entropy_spectrum(368,168,i,rng))

(4143306973, BitArray('0xddd8f5f6'))
(955241019, BitArray('0x3bd2ef38'))
(1276503626, BitArray('0x4ae6154c'))
(4009498663, BitArray('0x2718fcee'))
(2195688050, BitArray('0x728adf82'))
(859824952, BitArray('0x38e33f33'))
(2436222145, BitArray('0xc1cc3591'))
(2667970256, BitArray('0xd0fe059f'))
(3372662081, BitArray('0x41bd06c9'))
(2738999304, BitArray('0x08d041a3'))
(4235264195, BitArray('0xc30071fc'))
(3504191285, BitArray('0x35b7ddd0'))
(2158783448, BitArray('0xd86bac80'))
(3706267353, BitArray('0xd926e9dc'))
(2020726732, BitArray('0xccd77178'))
(1637089085, BitArray('0x3dff9361'))
(825020608, BitArray('0xc0d02c31'))
(1454531241, BitArray('0xa962b256'))
(4056381695, BitArray('0xff78c7f1'))
(2076483287, BitArray('0xd79ec47b'))


Best case c,k = 0,0; takes: 0.0s For 20 images: 0.5s  
Worst case c,k = 369,169; takes: 1m 24s For 20 images: 24m 12.4s  

In other words best case O(1), worst case: O(n^2)  
Average case: O(n^2)?  
  
Because of this generate a entropies based on only the first 841 frame sizes. (1-29 x 1-29)  